Notlarda one-vs-all OvA yı anlattık ama burada dümdüz yapıyor gibi görünebiliriz.  scikit-learn'de lojistik regresyon modelini çok sınıflı sınıflandırma (multi-class classification) problemleri için kullandığınızda, LogisticRegression sınıfı otomatik olarak one-vs-all (OvA) veya softmax regression (diğer adıyla multinomial) stratejisini uygular. Bu davranış, multi_class parametresine bağlıdır.  Ancak OvA veya softmax arasında bir tercih yapmak isterseniz, multi_class parametresini değiştirerek belirtebilirsiniz:

multi_class='ovr': One-vs-All kullanır.

multi_class='multinomial': Softmax Regression kullanır.

Solver, modelin kayıp fonksiyonunu minimize etmek için kullanılan algoritmadır. Logistic Regression gibi modellerde, solver, en iyi parametreleri bulmak için gradient descent veya benzeri optimizasyon yöntemlerini kullanır.

Scikit-learn'deki Logistic Regression için solver seçenekleri şunlardır:

lbfgs: L-BFGS (Limited-memory Broyden–Fletcher–Goldfarb–Shanno) algoritması. Büyük veri setleri için uygundur ve genellikle multinomial logistic regression için tercih edilir.

liblinear: Küçük veri setlerinde iyi performans gösteren bir doğrusal optimizasyon algoritması.

newton-cg: Newton yöntemiyle optimizasyon.

sag: Stochastic Average Gradient. Büyük ve yüksek boyutlu veri setleri için uygundur.

saga: SAG'ın iyileştirilmiş hali; elastik net gibi regularizasyonları destekler.

In [1]:
import pandas as pd
import seaborn as sns 
import matplotlib.pyplot as plt 
from sklearn.model_selection import train_test_split,cross_val_score
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import LabelEncoder,StandardScaler
from sklearn.metrics import confusion_matrix, accuracy_score

In [2]:
data = sns.load_dataset("iris")
data2 = data.copy()
data2.head()

,sepal_length,sepal_width,petal_length,petal_width,species
0,5.1,3.5,1.4,0.2,setosa
1,4.9,3.0,1.4,0.2,setosa
2,4.7,3.2,1.3,0.2,setosa
3,4.6,3.1,1.5,0.2,setosa
4,5.0,3.6,1.4,0.2,setosa


In [3]:
# eksik gözlem kontrolü
print(data2.isnull().sum())

sepal_length    0
sepal_width     0
petal_length    0
petal_width     0
species         0
dtype: int64


In [4]:
data2.info()
print(data2["species"].unique()) # 3 tane kategoriden oluşan bir bağımlı değişkenimiz varmış

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 150 entries, 0 to 149
Data columns (total 5 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   sepal_length  150 non-null    float64
 1   sepal_width   150 non-null    float64
 2   petal_length  150 non-null    float64
 3   petal_width   150 non-null    float64
 4   species       150 non-null    object 
dtypes: float64(4), object(1)
memory usage: 6.0+ KB
['setosa' 'versicolor' 'virginica']


In [5]:
# 2 den fazla kategorimiz olduğu için multiclass classification yani çoklu sınıflandırma
# problemine sahibiz. Değişkenlerimizi sayısal verilere dönüştürmek için encoding yapıyoruz.
encoder = LabelEncoder()
data2["species"] = encoder.fit_transform(data2["species"])
data2["species"].unique() # encoding yaptık 


array([0, 1, 2])

In [6]:
# veri kümemizi bölüyoruz
y= data2["species"]
X = data2.drop(columns="species",axis=1)

X_train, X_test, y_train, y_test = train_test_split(X,y,test_size=0.2,random_state=3)

In [7]:
# bağımsız değişkenlerimizi belirli bir standarda getirmek adına
# Standartlaştırma işlemi yapıyoruz. 
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.fit_transform(X_test)

In [8]:
model = LogisticRegression()
model.fit(X_train,y_train)
y_preds = model.predict(X_test)
cm = confusion_matrix(y_test,y_preds)
print(accuracy_score(y_test,y_preds))
print(cm)

0.9666666666666667
[[10  0  0]
 [ 0 10  0]
 [ 0  1  9]]


In [9]:
# normalde bu başarı oranı şüphenilmesi gereken bir başarı skoru o yüzden 
# genel olarak böyle bir sonuç alırsak Cross val score yapmak iyidir. 
# Bu veri seti çok genel ve her zaman yüksek doğruluk oranı dönen bir veriseti olduğu için
# normaldir ama yapmam gerekeni yapmalıyım
cval = cross_val_score(model,X_test,y_test,cv=10).mean()
print(cval) # yine aynı şekilde yüskek bir değer çıktı. 

0.9333333333333333


In [10]:
# OvA modeli
ova_model = LogisticRegression(multi_class='ovr', solver='liblinear')
ova_model.fit(X_train, y_train)
ova_preds = ova_model.predict(X_test)

# Multinomial modeli
multi_model = LogisticRegression(multi_class='multinomial', solver='lbfgs', max_iter=1000)
multi_model.fit(X_train, y_train)
multi_preds = multi_model.predict(X_test)

# Sonuçları karşılaştırma
print("OvA Accuracy:", accuracy_score(y_test, ova_preds))
print("Multinomial Accuracy:", accuracy_score(y_test, multi_preds))

OvA Accuracy: 0.9333333333333333
Multinomial Accuracy: 0.9666666666666667


c:\Users\Kemalettin\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\linear_model\_logistic.py:1256: FutureWarning: 'multi_class' was deprecated in version 1.5 and will be removed in 1.7. Use OneVsRestClassifier(LogisticRegression(..)) instead. Leave it to its default value to avoid this warning.
  warnings.warn(
c:\Users\Kemalettin\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\linear_model\_logistic.py:1247: FutureWarning: 'multi_class' was deprecated in version 1.5 and will be removed in 1.7. From then on, it will always use 'multinomial'. Leave it to its default value to avoid this warning.
  warnings.warn(
